In [2]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.offline as pyo
from scipy import signal
import scipy.cluster.hierarchy as spc
from pandas import read_excel
from ipywidgets import widgets
from ipywidgets import interactive, HBox, VBox
import plotly.io as pio

In [3]:
#Assume all CSVs are in same folder
PATH = "data_nico_quarterly/"
names = [name for name in os.listdir(PATH) if ".csv" in name]

In [4]:
def proc_cols(df, value_name):
    df = df.rename(columns={"Value": value_name})
    return df.drop(['INDICATOR', 'SUBJECT','MEASURE','FREQUENCY','Flag Codes'], axis=1)

def generate_df(csv_names,path):
    df = None
    str_end = len(".csv")
    for name in csv_names:
        if df is None:
            df = proc_cols(pd.read_csv(path+name),name[:-str_end])
        else:
            new_df = proc_cols(pd.read_csv(path+name),name[:-str_end])
            df = pd.merge(df, new_df,  how='outer', left_on=['LOCATION','TIME'], right_on = ['LOCATION','TIME'])
    return df

In [37]:
df = generate_df(names,PATH)

In [38]:
df_hur = df.drop(columns = ['GDP_growth']).replace({'-Q1':'-01-01','-Q2':'-04-01','-Q3':'-07-01','-Q4':'-10-01'}, regex=True)

In [39]:
df_hur = df_hur.pivot_table(values='HUR', index='TIME', columns='LOCATION')

In [40]:
def plot_lines(df,value_smooth):
    fig = go.Figure()
    for x in df.columns:
        if value_smooth == 0:
            fig.add_trace(go.Scatter(x=df.index, y=df[x], mode='lines',name=x))
        else:
            fig.add_trace(go.Scatter(x=df.index, y=signal.savgol_filter(df[x],value_smooth, 3), mode='lines',name=x))
    return fig

In [41]:
plot_lines(df_hur,0).show()